In [1]:
from ecmwf.opendata import Client
from datetime import datetime, timedelta

def download_latest_run(target, area=None):
    client = Client(model="aifs-single", source="ecmwf", resol="0p25")
    now = datetime.utcnow()
    date_str = now.strftime("%Y%m%d")
    # hours to try for latest available run, from most recent to older
    valid_hours = [18, 12, 6, 0]

    for h in valid_hours:
        try:
            params={
                "type":"fc", # forecast
                "stream":"oper", # operational stream
                "step": list(range(0, 49, 6)),  # forecasts every 6h up to +48h
                "param": ["tp", "10u", "10v", "tcc","q","tcw","sp","t"], # precipitation, 10m u and v wind, total cloud cover
                "date": date_str, # the most recent date
                "time": h, # the hour we are trying
                "target":target # output file name
            }
            client.retrieve(**params)
            print(f"Latest available run found: {date_str} {h:02d} UTC")
            return date_str, h
        except Exception as e:
            # if not, try the next hour
            continue

    # if none found, try yesterday 18UTC
    yesterday = (now - timedelta(days=1)).strftime("%Y%m%d")
    return yesterday, 18

_, _ = download_latest_run("forecast_globe.grib")


<multiple>:   0%|          | 0.00/160M [00:00<?, ?B/s]

Latest available run found: 20250829 00 UTC


In [147]:
from fmiopendata.wfs import download_stored_query
import requests
import xarray as xr
from datetime import timedelta
now = datetime.utcnow()
date_only=now.date()
# hours to try for latest available run, from most recent to older
valid_hours = [18, 12, 6, 0]

for h in valid_hours:
    origin_datetime = datetime.combine(date_only, datetime.min.time()) + timedelta(hours=h)    
    try:
        url = (
            "https://opendata.fmi.fi/download?"
            "producer=harmonie_scandinavia_surface&"
            "param=precipitation1h,windums,windvms,totalcloudcover&"
            f"origintime={origin_datetime.isoformat()}Z&"
            # f"starttime={starttime.isoformat()}Z&"
            # f"endtime={endtime.isoformat()}Z&"
            "bbox=5,54,31,72&"
            "projection=EPSG:4326&"
            "format=grib2&"
            "timestep=360&"   # 6 ساعتی
            "timesteps=9"     # 48 ساعت
        )

        print("Requesting GRIB data...")
        resp = requests.get(url)
        resp.raise_for_status()

        if not resp.content:
            raise RuntimeError("No GRIB data returned!")
        else:
            filename = "harmonie_scandinavia_latest.grib2"
            with open(filename, "wb") as f:
                f.write(resp.content)
                print("Saved:", filename)
    except Exception as e:
            # if not, try the next hour
            continue



Requesting GRIB data...
Requesting GRIB data...
Requesting GRIB data...
Saved: harmonie_scandinavia_latest.grib2
Requesting GRIB data...


In [14]:
from ecmwf.opendata import Client
from datetime import datetime, timedelta
import requests

def download_latest_run(target_global, target_scandinavia):
    client = Client(model="aifs-single", source="ecmwf", resol="0p25")
    now = datetime.utcnow()
    date_str = now.strftime("%Y%m%d")
    date_only=now.date()
    # hours to try for latest available run, from most recent to older
    valid_hours = [18, 12, 6, 0]

    for h in valid_hours:
        origin_datetime = datetime.combine(date_only, datetime.min.time()) + timedelta(hours=h)    
        try:
            # Get the regional data for Scandinavia
            url = (
                "https://opendata.fmi.fi/download?"
                "producer=harmonie_scandinavia_surface&"
                "param=PrecipitationAmount,windums,windvms,totalcloudcover&"
                f"origintime={origin_datetime.isoformat()}Z&"
                "bbox=5,54,31,72&"
                "projection=EPSG:4326&"
                "format=grib2&"
                "timestep=360&"   # 6 hours
                "timesteps=9"     # 48 hours
            )
            

            resp = requests.get(url)
            resp.raise_for_status()

            if not resp.content:
                raise RuntimeError("No Scandinavian NetCDF data returned!")
            else:
                print(f"Latest available run found: {date_str} {h:02d} UTC")
                with open(target_scandinavia, "wb") as f:
                    f.write(resp.content)
                    print("Scandinavia Saved:", target_scandinavia)
            # Get the global data
            params = {
                "type":"fc", # forecast
                "stream":"oper", # operational stream
                "step": list(range(0, 49, 6)),  # forecasts every 6h up to +48h
                "param": ["tp", "10u", "10v", "tcc"], # precipitation, 10m u and v wind, total cloud cover
                "date": date_str, # the most recent date
                "time": h, # the hour we are trying
                "target":target_global # output file name
                
            }
            client.retrieve(**params)
            print(f"Latest available run found: {date_str} {h:02d} UTC")
            print("Global Saved:", target_global)
            return date_str, h
        except Exception as e:
            # if not, try the next hour
            continue

    # if none found, try yesterday 18UTC
    yesterday = (now - timedelta(days=1)).strftime("%Y%m%d")
    return yesterday, 18

_, _ = download_latest_run("forecast_globe.grib", "forecast_scandinavia.grib")

Latest available run found: 20250829 06 UTC
Scandinavia Saved: forecast_scandinavia.grib


<multiple>:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

Latest available run found: 20250829 06 UTC
Global Saved: forecast_globe.grib


In [15]:
xr.open_dataset("forecast_scandinavia.grib", engine="cfgrib")

skipping variable: paramId==228164 shortName='tcc'
Traceback (most recent call last):
  File "/home/motlaghz/.conda/envs/myenv/lib/python3.9/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/motlaghz/.conda/envs/myenv/lib/python3.9/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='entireAtmosphere' value=Variable(dimensions=(), data=0.0) new_value=Variable(dimensions=(), data=10.0)


<xarray.Dataset>
Dimensions:            (step: 9, latitude: 801, longitude: 1157)
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 0 days 00:00:00 ... 2 days 00:0...
    entireAtmosphere   float64 ...
  * latitude           (latitude) float64 54.0 54.02 54.05 ... 71.95 71.98 72.0
  * longitude          (longitude) float64 5.0 5.022 5.045 ... 30.96 30.98 31.0
    valid_time         (step) datetime64[ns] ...
    heightAboveGround  float64 ...
Data variables:
    rain_con           (step, latitude, longitude) float32 ...
    u10                (step, latitude, longitude) float32 ...
    v10                (step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-29T15:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [19]:
import xarray as xr
def merge_datasets(filename):
    # Open individual datasets for each parameter and merge them
    tp = xr.open_dataset(filename, engine="cfgrib",filter_by_keys={'shortName': 'rain_con'}, backend_kwargs={"indexpath": ""})
    u10 = xr.open_dataset(filename, filter_by_keys={'shortName': '10u'}, backend_kwargs={"indexpath": ""})
    v10 = xr.open_dataset(filename, filter_by_keys={'shortName': '10v'}, backend_kwargs={"indexpath": ""})
    tcc = xr.open_dataset(filename, filter_by_keys={'shortName': 'tcc', 'typeOfLevel': 'entireAtmosphere'}, backend_kwargs={"indexpath": ""})

    
    tp.close()
    u10.close()
    v10.close()
    tcc.close()
    
    return tp, u10, v10, tcc
tp, u10, v10, tcc = merge_datasets("forecast_scandinavia.grib")

In [20]:
with xr.open_dataset("forecast_globe.grib", engine="cfgrib", backend_kwargs={"indexpath": ""}) as ds1, xr.merge([tp, u10, v10, tcc],compat="override") as ds2:
    # plot_all_parameters(ds1, ds2)
    print(ds2['rain_con'])
    print(ds1['tp'])

<xarray.DataArray 'rain_con' (step: 9, latitude: 801, longitude: 1157)>
[8340813 values with dtype=float32]
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 0 days 00:00:00 ... 2 days 00:0...
    entireAtmosphere   float64 ...
  * latitude           (latitude) float64 54.0 54.02 54.05 ... 71.95 71.98 72.0
  * longitude          (longitude) float64 5.0 5.022 5.045 ... 30.96 30.98 31.0
    valid_time         (step) datetime64[ns] ...
    heightAboveGround  float64 ...
Attributes: (12/29)
    GRIB_paramId:                             201113
    GRIB_dataType:                            af
    GRIB_numberOfPoints:                      926757
    GRIB_typeOfLevel:                         entireAtmosphere
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                surface precipitation amount, r...
    GR

In [ ]:

import xarray as xr
import matplotlib.pyplot as plt
import subprocess

# Example using ecCodes
# subprocess.run(["grib_to_netcdf", "forecast_globe.grib", "-o", "forecast_globe.nc"])
ds_scandinavia = xr.open_dataset("forecast_scandinavia.nc")
# ds_global = xr.open_dataset("forecast_globe.nc")
print(ds_global.dims)
ds_scandinavia = ds_scandinavia.drop_vars("time")
print(ds_scandinavia.dims)


Frozen({'longitude': 1440, 'latitude': 721, 'time': 9})
Frozen({'lat': 801, 'lon': 1157, 'time_h': 9, 'time_bounds': 2, 'time': 9})


In [44]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from ipywidgets import interact, widgets

def plot_all_parameters(ds):
    parameters = ["all", "tp", "wind", "tcc"]
    regions = ["global", "nordic"]

    def plot_map(parameter="all", region="global"):
        time_str = str(ds["tp"].coords["valid_time"].values)

        # Determine extent
        if region.lower() == "nordic":
            extent = [5, 32, 55, 72]
        else:
            extent = [ds.longitude.min().item(), ds.longitude.max().item(),
                      ds.latitude.min().item(), ds.latitude.max().item()]

        # Select the sub-region
        lon_min, lon_max, lat_min, lat_max = extent
        if ds.latitude[0] > ds.latitude[-1]:
            ds_sel = ds.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))
        else:
            ds_sel = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

        # Decide how many subplots
        if parameter == "all":
            nplots = 3
            params_to_plot = ["tp", "wind", "tcc"]
        else:
            nplots = 1
            params_to_plot = [parameter]

        fig, axes = plt.subplots(nplots, 1, figsize=(10, 6*nplots),
                                 subplot_kw={'projection': ccrs.PlateCarree()})
        # If only one plot, make axes iterable
        if nplots == 1:
            axes = [axes]

        for ax, p in zip(axes, params_to_plot):
            ax.set_extent(extent, crs=ccrs.PlateCarree())
            ax.add_feature(cfeature.COASTLINE)
            ax.add_feature(cfeature.BORDERS)

            if p == "tp":
                ds_sel["tp"].plot(ax=ax, cmap="Blues", transform=ccrs.PlateCarree(),
                                  cbar_kwargs={"label": "Total Precipitation (m)", "pad": 0.02})
                ax.set_title(f"Precipitation at {time_str} ({region})")

            elif p == "wind":
                skip = (slice(None, None, 3), slice(None, None, 3)) if region.lower() == "nordic" else (slice(None, None, 10), slice(None, None, 10))
                lats = ds_sel["latitude"].values[skip[0]]
                lons = ds_sel["longitude"].values[skip[1]]
                u = ds_sel["u10"].values[skip]
                v = ds_sel["v10"].values[skip]
                Lon, Lat = np.meshgrid(lons, lats)
                wind_speed = np.sqrt(u**2 + v**2)
                scale = 700 if region=="global" else 150
                q = ax.quiver(Lon, Lat, u, v, wind_speed, cmap="coolwarm",
                              transform=ccrs.PlateCarree(), scale=scale)
                cbar = plt.colorbar(q, ax=ax, orientation="vertical", pad=0.02)
                cbar.set_label("Wind speed (m/s)")
                ax.set_title(f"Wind at {time_str} ({region})")

            elif p == "tcc":
                ds_sel["tcc"].plot(ax=ax, cmap="bone", transform=ccrs.PlateCarree(),
                                   cbar_kwargs={"label": "Total Cloud Cover (fraction)", "pad": 0.02})
                ax.set_title(f"Cloud Cover at {time_str} ({region})")

        plt.show()

    interact(plot_map,
             parameter=widgets.Dropdown(options=parameters, value="all", description="Parameter:"),
             region=widgets.Dropdown(options=regions, value="global", description="Region:"))


In [45]:
plot_all_parameters(ds_globe)

interactive(children=(Dropdown(description='Parameter:', options=('all', 'tp', 'wind', 'tcc'), value='all'), D…